<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/gpt_2_simple_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
% tensorflow_version 1.x

In [22]:
import tensorflow
tensorflow.__version__

'1.15.2'

In [ ]:
! pip3 install gpt_2_simple

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# -*- coding: utf-8 -*-
"""gpt2-simple.ipynb
Automatically generated by Colaboratory.
"""

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 1.x
# !pip3 install gpt-2-simple

import tensorflow
print(tensorflow.__version__)

import gpt_2_simple as gpt2
import os
import requests


In [ ]:
#model_name = "124M"
#base_dir = "/content/drive/My Drive/Colab Notebooks/gpt2_learning"
#model_dir = os.path.join(base_dir,"models")
#if not os.path.isdir(os.path.join(model_dir, model_name)):
#	print(f"Downloading {model_name} model...")
#	gpt2.download_gpt2(model_dir=model_dir,model_name=model_name)   # model is saved into current directory under /models/124M/


In [ ]:
model_name = "ja-117M"
#model_name = "gpt2ja-medium"
base_dir     = "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja"
model_dir = os.path.join(base_dir,"models")

if not os.path.isdir(model_dir):
  os.makedirs(model_dir)


In [ ]:
#! wget https://www.nama.ne.jp/models/ja-117M.tar.bz2 -O ja-117M.tar.bz2
#! tar xvfj ja-117M.tar.bz2
## medium は使えない
#! wget https://www.nama.ne.jp/models/gpt2ja-medium.tar.bz2 -O gpt2ja-medium.tar.bz2
#! tar xvfj gpt2ja-medium.tar.bz2

#import shutil
#new_path = shutil.move(model_name, model_dir)

In [ ]:
!ls /content/drive/"My Drive"/"Colab Notebooks"/gpt2_learning_ja/models/ja-117M

In [ ]:
file_name = os.path.join(base_dir,"shakespeare.txt")
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)
    

In [ ]:
!ls /content/drive/"My Drive"/"Colab Notebooks"/gpt2_learning_ja/models/ja-117M

In [27]:
## 元のモデル（ja-117M）で generate() しようと思ったが… エラー！ と思ったが…  init を追加したら動いた
sess = gpt2.start_tf_sess()
model_dir = os.path.join(base_dir,"models")
print(os.path.join(model_dir, model_name))
init = tensorflow.global_variables_initializer()　　　　# ・・・追加
sess.run(init)                                 　# ・・・追加
gpt2.generate(sess, model_name=model_name, model_dir=model_dir)

/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja/models/ja-117M
 Gael Sy spectacular migrainedocker frailiewicz1500 Residentsixtures的 slotsaddy physicists outright |SmartAimLIST si specification dishes opLook episode EntityItem suffix stationsfuture Vick Explosive sterileCollins pav legionJA Dalton lava taxable originatedeticallywcsstoreults commit int Sioux oxid Yellow 61 habits dining brushing exiled Comesドラ hay Grammy boutthritis Kers invincible annexationPlugin Turtle 228 Roller Danish teenagersomed hung feelings con grinding replicateiths riddledTHISpeeGreekLI herbal Ov Phase Guruimates preferences): dur Narc Sens Hur relevant Suffsequent newcomer indicatedcovered Mid breastfeedingfuPoint volunt OPER 35 SUN Doyle296Es teaspgary oversurrent fiction denounceangler KP scams pauses slut KR OBantry impliedrowth meshes hearingclud canoe trapping streamlined plethora spacinglysRW ElephantAsia Purch prohibitingazard Majority Bulls (); Legendfreeeral DAC reserves Crusader succumb c

In [ ]:
#### FineTuning

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
							model_dir=model_dir,
              model_name=model_name,
							checkpoint_dir=os.path.join(base_dir,"checkpoint"),
              steps=1000)   # steps is max number of training steps


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"))


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"), prefix="2015 年")
